In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.linalg as linalg
import scipy.stats as stats
import scipy.interpolate as interp
import scipy.signal as signal
from scipy.optimize import curve_fit
from scipy.optimize import minimize 

import warnings
from scipy.linalg import toeplitz
warnings.filterwarnings('ignore')
%pylab inline 

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 22}

axes = {'titlesize'  : 22,
        'labelsize'  : 22}

legend = {'fontsize'  : 22}

figure = {'figsize'  : (10,5)}

matplotlib.rc('font', **font)
matplotlib.rc('axes', **axes)
matplotlib.rc('legend', **legend)
matplotlib.rc('figure', **figure)

# Read in worm GFP

In [ ]:
### Read in worm data from SharedData .npz file
Worm = 'GFP'
Centerline = True
Neuron_Position = False
Close = False

worm_data = np.load('../SharedData/Worm'+Worm+'.npz')
print('The loaded Primary npz contains the variables:\n', np.sort([i for i in worm_data]))

if Worm == 'GFP':
    G_sig = 'GFP' 
else:
    G_sig = 'GCaMP'
    from scipy.interpolate import interp1d 


### Import desired variables
G_Raw = worm_data['G_Raw']
R_Raw = worm_data['R_Raw']
Time = worm_data['Time']


### Import Centerline Data?
if Centerline:
    CLdata = np.load('../SharedData/Worm_Angles/WormAngle'+Worm+'.npz')
    print('The loaded Centerline npz contains the variables:\n', np.sort([i for i in CLdata]))
    
    CL_PCs = CLdata['proj_neural_thetas'].T


### Import Neuron Position Data?
if Neuron_Position:
    if Close:
        NPdata = np.load('../SharedData/NPos_PCA/Worm'+Worm+'_NPosPCA_Close.npz')
        print("Close neurons with no tail was loaded")
    else:
        NPdata = np.load('../SharedData/NPos_PCA/Worm'+Worm+'_NPosPCA.npz')
    print('The loaded Neuron Position npz contains the variables:\n', np.sort([i for i in NPdata]))
    
    NP_PCs = NPdata['NP_PCs']

## Process worm data

In [ ]:
### Determine which time columns have NaNs pick first and last w/o any NaNs
badcols = np.array([x|y for (x,y) in zip(np.isnan(G_Raw).any(axis=0), np.isnan(R_Raw).any(axis=0))])
begin_col = np.arange(0,len(Time))[~badcols][0]
end_col = np.arange(0,len(Time))[~badcols][-1] + 1

### Shave off times from before first and last full column
G_Raw = G_Raw[:,begin_col:end_col]
R_Raw = R_Raw[:,begin_col:end_col]
Time = Time[begin_col:end_col, 0]

if Centerline: CL_PCs = CL_PCs[:,begin_col:end_col]
if Neuron_Position: NP_PCs = NP_PCs[:,begin_col:end_col]
    
    
[neuron_length, neuron_time] = np.shape(G_Raw)
print('Neurons:', neuron_length, '\nTime Points:', neuron_time, '\nFrom', Time[0], 's to', Time[-1], 's')

### Fill in NaNs with interpolation
for i in np.arange(len(G_Raw)):
    
    g_bad = np.isnan(G_Raw[i])
    if g_bad.any():
        if Worm == 'GFP':
            g_interp = interp.interp1d(Time[~g_bad], G_Raw[i,~g_bad], kind='cubic', assume_sorted=True)
        else:
            g_interp = interp1d(Time[~g_bad], G_Raw[i,~g_bad], kind='cubic', assume_sorted=True)
        G_Raw[i][g_bad] = g_interp(Time[g_bad])
    
    r_bad = np.isnan(R_Raw[i])
    if r_bad.any():
        if Worm == 'GFP':
            r_interp = interp.interp1d(Time[~r_bad], R_Raw[i,~r_bad], kind='cubic', assume_sorted=True)
        else:
            r_interp = interp1d(Time[~r_bad], R_Raw[i,~r_bad], kind='cubic', assume_sorted=True)
        R_Raw[i][r_bad] = r_interp(Time[r_bad])
    
    #print(i)
    
    ### Visualize interpolated points
#     plt.scatter(Time[~r_bad], R_Raw[i,~r_bad], color='blue')
#     plt.plot(Time, R_Raw[i], color='blue', alpha=0.2)
#     plt.scatter(Time[r_bad], R_Raw[i,r_bad], color='red')
#     plt.show()
#     break

Autocovariance
$\hat{\gamma}_x(h) = \frac{1}{n}\sum_{i=1}^{n-|h|}\left(x_i-\bar{x}\right)\left(x_{i+|h|} - \bar{x}\right)$

$C(\tau) = E[\tilde r(t) * \tilde r(t+\tau)]$

where $\tilde r$ is the zero-centered r(t), i.e., 
$\tilde r(t) = r(t) - E[r(t)]$.

# Fit the distribution to neurons

## Trying on many neurons

In [ ]:
import scipy.io as sio
n9 = R_Raw[9,:]
n12 = R_Raw[12,:]
sio.savemat('neurons9_12.mat',{'n9':n9,'n12':n12})

In [ ]:
from simulate_activity import calc_corr
from est_param import *

In [ ]:
def fit_neurons(neurons, LAGS = 300, mse_res_all = [], mle_res_all = []):
    f, ax = plt.subplots(len(neurons),4,figsize=(20,5*len(neurons)))

    for i,v in enumerate(neurons):
        print(v)
        # Make R corr
        R = np.copy(R_Raw[v,:])
        # unit standard deviation
        std0 = std(R)
        R = np.divide(R,std0)
        # mean centered
        R = R - np.mean(R)
        (xx, R_t_corr) = calc_corr(R, R, LAGS)

        # MSE
        inf_params = est_MSE_R(xx,R_t_corr)
        mse_res_all.append(inf_params)

        fitted_MSE_R = create_fitted(xx,inf_params)
        
        if len(neurons)>1:
            ax[i,0].plot(xx,R_t_corr,label='Real')
            ax[i,0].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[i,0].set_title("N %d: Rho: %1.2f L: %1.2f P: %1.2f sig_r: %1.2f" 
                  % (v, inf_params[0],inf_params[1],inf_params[2],inf_params[3]))
            ax[i,0].legend()
        else:
            ax[0].plot(xx,R_t_corr,label='Real')
            ax[0].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[0].set_title("N %d: Rho: %1.2f L: %1.2f P: %1.2f sig_r: %1.2f" 
                  % (v, inf_params[0],inf_params[1],inf_params[2],inf_params[3]))
            ax[0].legend()

        # MLE
        mle_res = est_MLE_R(R, inf_params,toPrint = False,method='SLSQP')
        print(mle_res)
        mle_res_all.append(mle_res)

        fitted_MLE_R = create_fitted(xx,mle_res)
        if len(neurons)>1:
            ax[i,1].plot(xx,R_t_corr,label='Real')
            ax[i,1].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[i,1].set_title("N %d: Rho: %1.2f L: %1.2f\nP: %1.2f sig_r: %1.2f" 
                  % (v, mle_res[0],mle_res[1],mle_res[2],mle_res[3]))
            ax[i,1].legend()

            ax[i,2].plot(xx,R_t_corr,label='Real')
            ax[i,2].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[i,2].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[i,2].legend()

            ax[i,3].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[i,3].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[i,3].legend()
        else:
            ax[1].plot(xx,R_t_corr,label='Real')
            ax[1].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[1].set_title("N %d: Rho: %1.2f L: %1.2f\nP: %1.2f sig_r: %1.2f" 
                  % (v, mle_res[0],mle_res[1],mle_res[2],mle_res[3]))
            ax[1].legend()

            ax[2].plot(xx,R_t_corr,label='Real')
            ax[2].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[2].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[2].legend()

            ax[3].plot(xx,fitted_MSE_R,label='MSE Fit')
            ax[3].plot(xx,fitted_MLE_R,label='MLE Fit')
            ax[3].legend()

In [ ]:
mse_res_all = []
mle_res_all = []

### Order of neurons for MLE ascending GFP

array([39,  3, 24, 12, 29, 21, 50,  9,  6, 13, 30, 42, 44, 18, 62, 45, 33,
        4, 19, 49, 31, 55, 14, 15, 28, 48, 47, 10, 46, 60, 11, 58, 27, 20,
       34, 57, 22,  5, 26, 40, 25, 36,  0, 61, 52, 38, 59, 43, 37,  2, 17,
       16, 56, 35, 54, 23,  8, 41, 51, 32,  1, 53,  7])

In [ ]:
range(neuron_length)

In [ ]:
import timeit
neurons =np.array(range(neuron_length))
small_mse_neurons = [39,3,24]
large_mse_neurons = [1,53,7]

In [ ]:
#neurons = np.concatenate((small_mse_neurons, large_mse_neurons))
def test():
    fit_neurons(neurons, 300, mse_res_all, mle_res_all)
print(timeit.timeit(test,number=1)/(1*len(neurons)))

# 3. 
Plug in $\hat{R}_m(\tau), \hat{\mu}_m$ into $R_r(\tau) = R_m(\tau) + \sigma_r^2\delta(\tau)$, and $R_{r,g}(\tau) = 
\mu_mR_{a,m}(\tau) + \mu_aR_m(\tau)$ and use these equations to obtain $\hat{R}_{a,m}(0), \mu_m$ by:
  $$\left(\begin{matrix}R_{a,m}(0)\\\mu_a\\\end{matrix}\right) = \left(\begin{matrix} 1&\mu_m\\\mu_m&R_m(0)\\\end{matrix}\right)^{-1}\left(\begin{matrix}\mu_g\\R_{r,g}(0)\\\end{matrix}\right)$$
  
# 4. 
  Set $\hat{R}_{a,m}(\tau) = \mu_m^{-1}\left(R_{r,g}(\tau) - \mu_aR_m\left(\tau\right)\right)$

In [ ]:
LAG = 50

In [ ]:
r_am0_all = np.empty((len(neurons),1))
mu_a_all = np.empty_like((r_am0_all))
for i,n in enumerate(neurons):
    # set up variables
    mle_res = mle_res_all[0]
    R = np.copy(R_Raw[n,:])
    G = np.copy(G_Raw[n,:])
    (xx,r_rg) = calc_corr(R,G,LAG)
    mu_r = mean(R)
    mu_g = mean(G)

    # Let's take that equation to be C = matrixA*matrixB
    mu_m = mu_r
    r_m0 = mle_res[0]
    matrixA = np.linalg.inv(np.reshape([1,mu_m,mu_m,r_m0],(2,2)))
    r_rg0 = r_rg[int(len(r_rg)/2)]
    matrixB = np.reshape([mu_g,r_rg0],(2,1))
    matrixC = matrixA.dot(matrixB)
    r_am0_all[i] = matrixC[0,0]
    mu_a_all[i] = matrixC[1,0]
    
    R_m = mu_m
    r_am = (r_rg - mu_a*R_m)/mu_m 
    plt.plot(xx,r_am,label=str(n))
    
print('r_am0: mean: %1.3f, var: %1.3f' % (mean(r_am0_all), var(r_am0_all)))
print('mu_a: mean: %1.3f, var: %1.3f' % (mean(mu_a_all), var(mu_a_all)))
plt.title("R_am")
plt.legend()

5. Fit parameteric form of $R_a\left(\tau\right)$ and $\sigma_g^2$ by equation 29

$$R_g = R_aR_m+R_{am}R_{ma}+\mu_a^2R_m + \mu_m^2R_a + \sigma_g^2\delta$$